# **1. Perkenalan Dataset**

Dataset adalah data primer yang saya kumpulkan sendiri. Data berupa nilia Mahasiswa Universitas Tanjungpura. Data ini terbagi atas Program Studi Sistem Informasi & Rekayasa Sistem Komputer, yang masing-masing memiliki data dari angkatan 2021 sampai 2024. Isi datanya Mencangkup:
  
1. matkul: "Bahasa Indonesia" (judul mata kuliah)
2. dosen: "MARIYADI, M.Pd" (nama dosen)
3. no: 1 (nomor angka urutan)
4. nim: "H1051211002" (nomor induk mahasiswa)
5. mahasiswa: "DEVIN SANYOKA" (nama mahasiswa)
6. nilai: "88.30"
7. nilai_angka: "4.00"
8. nilai_huruf: "A"
9. prodi: "rsk" | "si" (Rekayasa Sistem Komputer atau Sistem Informasi)

# **2. Import Library**

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import os

# **3. Memuat Dataset**

In [ ]:
# Dapatkan direktori kerja saat ini (untuk Jupyter Notebook)
base_dir = os.getcwd()

# Gabungkan path relatif file CSV
file_path = os.path.join(base_dir, "./../nilai_mahasiswa_raw.csv")
print(f"✅ File CSV: {file_path}")
# Load dataset
df = pd.read_csv(file_path)

df.head()

✅ File CSV: c:\Users\ADVAN\Documents\Dicoding\msml\SMSML_LeoPrangsT\preprocessing\./../nilai_mahasiswa_sisfo_resiskom_2021-2024.csv


,matkul,dosen,no,nim,mahasiswa,nilai,nilai_angka,nilai_huruf,prodi
0,Bahasa Indonesia,"MARIYADI, M.Pd",1,H1051211002,DEVIN SANYOKA,88.30,4.0,A,rsk
1,Bahasa Indonesia,"MARIYADI, M.Pd",2,H1051211004,FELISITAS MARETTANIA,88.30,4.0,A,rsk
2,Bahasa Indonesia,"MARIYADI, M.Pd",3,H1051211006,MUHAMMAD FAISHAL HADY,86.20,4.0,A,rsk
3,Bahasa Indonesia,"MARIYADI, M.Pd",4,H1051211008,MAULIDIA,83.80,4.0,A,rsk
4,Bahasa Indonesia,"MARIYADI, M.Pd",5,H1051211010,JOSIA WISE SITINJAK,85.90,4.0,A,rsk


**Insight:** Model yang dibuat adalah collaborative filtering, data yang dipakai adalah **nim** sebagai label, **nilai_angka** sebagai parameter, dan **mata_kuliah** sebagai target.

# **4. Exploratory Data Analysis (EDA)**

In [7]:
# Melihat jumlah nim dan jumlah matkul
print(f"{df['nim'].nunique()} nim, {df['matkul'].nunique()} maktul")

print('\n--------- Null Values ---------')
print(df[['matkul','nim','nilai_angka']].isna().sum())

# print('\n--------- Invalid Values ---------')
for col in ['nilai_angka']: # kolom ordinal yang perlu diperiksa
    print(f"{col}: {df[col].unique().tolist()}")

# check for duplicate 'nim' and 'matkul' pairs
duplicate_nim_matkul = df[df.duplicated(subset=['nim', 'matkul'], keep=False)]
print("Jumlah pasangan 'nim' dan 'matkul' yang memiliki duplikasi:", duplicate_nim_matkul.shape[0])
display(duplicate_nim_matkul.sort_values(by='nim').head())

1397 nim, 211 maktul

--------- Null Values ---------
matkul            0
nim               0
nilai_angka    1216
dtype: int64
nilai_angka: ['4.0', '0.0', '3.5', '3.0', '2.0', '2.5', '1.0', '1.5', nan, '4', '0', '2025-05-03 00:00:00', '3', '2', '1', '2025-05-02 00:00:00', '2025-05-01 00:00:00']
Jumlah pasangan 'nim' dan 'matkul' yang memiliki duplikasi: 3648


,matkul,dosen,no,nim,mahasiswa,nilai,nilai_angka,nilai_huruf,prodi
10554,Tugas Akhir,-,2,H1051161003,ZUBAIR RASYID HS,NaN,3,B,rsk
2801,Tugas Akhir,-,47,H1051161003,ZUBAIR RASYID HS,NaN,NaN,NaN,rsk
10555,Tugas Akhir,-,3,H1051161004,NUR FITRIANA PUTRI,NaN,2025-05-03 00:00:00,B+,rsk
2802,Tugas Akhir,-,48,H1051161004,NUR FITRIANA PUTRI,NaN,NaN,NaN,rsk
10556,Tugas Akhir,-,4,H1051161015,ADHETYA PUTRA PERDANA,NaN,3,B,rsk


**insight:** data duplikasi (**nim** & **matkul**) dan invalid **nilai_angka** perlu di hapus atau konversi

# **5. Data Preprocessing**

In [8]:
df2 = df.copy(deep=True) # Berganti ke versi 2 (lebih bersih)

# --- CLEANING ---
# Hapus kolom yang tidak diperlukan
df2 = df2.drop(columns=['dosen', 'no', 'mahasiswa', 'nilai', 'nilai_huruf', 'prodi'])

# Convert 'nilai_angka' to numeric, coercing errors to NaN
df2['nilai_angka'] = pd.to_numeric(df2['nilai_angka'], errors='coerce')

# Sort by 'nilai_angka' (NaNs will be at the end) and then drop duplicates
# This keeps the row with a valid numeric value if it exists, otherwise keeps the first row
df2 = df2.sort_values(by=['nim', 'matkul', 'nilai_angka'], na_position='last')
df2.drop_duplicates(subset=['nim', 'matkul'], keep='first', inplace=True)

# Drop rows where 'nilai_angka' is still NaN after duplicate removal
df2.dropna(subset=['nilai_angka'], inplace=True)

# Convert 'nilai_angka' to float
df2['nilai_angka'] = df2['nilai_angka'].astype(float)

# -- after transform check --
print(f"{df2['nim'].nunique()} nim, {df2['matkul'].nunique()} maktul")
print(df2[['matkul','nim','nilai_angka']].isna().sum())
print(f"nilai_angka: {df2['nilai_angka'].unique().tolist()}")
# check for duplicate 'nim' and 'matkul' pairs
duplicate_nim_matkul = df2[df2.duplicated(subset=['nim', 'matkul'], keep=False)]
print("Jumlah pasangan 'nim' dan 'matkul' yang memiliki duplikasi:", duplicate_nim_matkul.shape[0])

1348 nim, 211 maktul
matkul         0
nim            0
nilai_angka    0
dtype: int64
nilai_angka: [4.0, 3.5, 2.0, 3.0, 2.5, 0.0, 1.0, 1.5]
Jumlah pasangan 'nim' dan 'matkul' yang memiliki duplikasi: 0


In [9]:
# Encode mahasiswa dan mata kuliah
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

df2['user'] = user_encoder.fit_transform(df2['nim'])
df2['item'] = item_encoder.fit_transform(df2['matkul'])

# Normalisasi nilai_angka menjadi 0-1
scaler = MinMaxScaler()
df2['rating'] = scaler.fit_transform(df2[['nilai_angka']])

# Dataset siap untuk NCF
print(df2[['user', 'item', 'rating']].head())

      user  item  rating
2755     0   208   1.000
2756     1   208   0.875
782      2    51   0.500
2757     2   208   0.750
2758     3   208   0.750


# **6. Export** (di luar template)

In [ ]:
# # Simpan hasil preprocess
df2.to_csv('nilai_mahasiswa-preprocessed.csv', index=False)

print("✅ Data hasil preprocessing berhasil disimpan sebagai 'nilai_mahasiswa-preprocessed.csv'")

✅ Data hasil preprocessing berhasil disimpan sebagai 'nilai_mahasiswa-preprocessed-ipynb.csv'
